In [1]:
from brightway2 import *
from bw2data.parameters import *


In [2]:
projects.delete_project('testparameters')

'default'

In [3]:
projects.set_current('testparameters')
databases.clear()

kwargs: {} <class 'dict'>
name: testparameters <class 'str'>


In [4]:
parameters

Parameters manager with 0 objects

In [5]:
t_db = Database('SAtestdb')

t_db.write({
    ("SAtestdb", "Electricity production"):{
        'name':'Electricity production',
        'unit': 'kWh', 
        'exchanges': [{
                'input': ('SAtestdb', 'Carbon dioxide'),
                'amount': 1,
                'unit': 'kg',
                'type': 'biosphere' # one parameter here
            },{
                'input': ('SAtestdb', 'Sulphur dioxide'),
                'amount': 1, # same value
                'unit': 'kg',
                'type': 'biosphere',
                'formula': 'parSO2' # other parameter here
            },{
                'input': ('SAtestdb', 'Electricity production'), #important to write the same process name in output
                'amount': 1,
                'unit': 'kWh',
                'type': 'production'
            }]
        },
    ('SAtestdb', 'Carbon dioxide'):{'name': 'Carbon dioxide', 'unit':'kg', 'type': 'biosphere'},
    ('SAtestdb', 'Sulphur dioxide'):{'name': 'Sulphur dioxide', 'unit':'kg', 'type': 'biosphere'}
   
    })

myLCIAdata = [[('SAtestdb', 'Carbon dioxide'), 1.0], 
              [('SAtestdb', 'Sulphur dioxide'), 1.0]]

imaginaryLCIAmethod = ('just', 'another', 'LCIAmethod')
my_method = Method(imaginaryLCIAmethod)
my_method.validate(myLCIAdata)
my_method.register() 
my_method.write(myLCIAdata)
my_method.load()


lca = LCA({t_db.get('Electricity production') : 1}, imaginaryLCIAmethod)
lca.lci()
lca.lcia()
print('-----\n-----\n-----\n-----\nlca score =', lca.score)

Writing activities to SQLite3 database:
0%  100%
[###] | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 05/10/2019 15:36:31
  Finished: 05/10/2019 15:36:31
  Total time elapsed: 00:00:00
  CPU %: 55.50
  Memory %: 0.75
-----
-----
-----
-----
lca score = 2.0


In [6]:
databases

Databases dictionary with 1 object(s):
	SAtestdb

In [7]:
ep = Database('SAtestdb').get('Electricity production')

In [11]:
ep_co2 = list(ep.exchanges())[0]

In [15]:
ep_co2['formula'] = 'parCO2'

In [16]:
ep_co2.save()

In [ ]:
ep.

In [18]:
ep_co2.as_dict()

{'amount': 1,
 'formula': 'parCO2',
 'input': ('SAtestdb', 'Carbon dioxide'),
 'output': ('SAtestdb', 'Electricity production'),
 'type': 'biosphere',
 'unit': 'kg'}

In [19]:
# Step 3. Define the details of the parameter 'my_parameter'
activity_data2 = [{'name': 'parCO2', 'amount': 1.0, 'database': 'SAtestdb', 'code' : 'parCO2code'},
                 {'name': 'parSO2', 'amount': 1.0, 'database': 'SAtestdb', 'code' : 'parSO2code'}]


    

In [20]:
parameters.new_activity_parameters(activity_data2, "my group 2", overwrite=True)


In [21]:
parameters

Parameters manager with 2 objects

In [22]:
parameters.add_exchanges_to_group("my group 2", t_db.get('Electricity production'))


2

In [23]:
parameters

Parameters manager with 3 objects

In [ ]:
ActivityParameter.recalculate_exchanges("my group 2")

In [ ]:
from SALib.sample import saltelli
from SALib.analyze import sobol

In [ ]:
problem = { 'num_vars': 2, 
            'names': ['parCO2', 'parSO2'], 
            'bounds': [[0.9, 1.1], [0.9, 1.1]] }  # 10 % variation

# Step 2. Generate the samples
param_values = saltelli.sample(problem, 5)
print(param_values.shape)
param_values[0:5]

In [ ]:
import numpy as np

In [ ]:
GSA_el_value_results = []
for v in param_values:
    ActivityParameter.update(amount = v[0]).where(ActivityParameter.name == 'parCO2').execute()
    ActivityParameter.update(amount = v[1]).where(ActivityParameter.name == 'parSO2').execute()
    
    ActivityParameter.recalculate_exchanges("my group 2")
    
    lca = LCA({t_db.get('Electricity production') : 1}, imaginaryLCIAmethod)
    lca.lci()
    lca.lcia()
    GSA_el_value_results.append(lca.score)

# Step 4. Feed the problem and the results to the Sobol function to obtain the Sobol indices 
Si = sobol.analyze(problem, np.array(GSA_el_value_results), print_to_console=True) # must use np.array
# Returns a dictionary with keys 'S1', 'S1_conf', 'ST', and 'ST_conf'
# (first and total-order indices with bootstrap confidence intervals)